In [1]:
import requests
from bs4 import BeautifulSoup
# Make a request to the website
r = requests.get('https://www.bbc.com')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':"module__content"}).find_all('a'):
    i['href'] = "https://www.bbc.com"+i['href'] + '?page=all'
    link.append(i['href'])
new_link = []
for i in range(0, len(link), 3):
    new_link.append(link[i])
print(new_link)
del new_link[3:4]
print(new_link)
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in new_link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'ssrcss-7uxr49-RichTextContainer e5tfeyi1'}).find_all('p'):
        sen.append(i.text)

    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))
    print(documents)

['https://www.bbc.com/news/world-middle-east-64150409?page=all', 'https://www.bbc.com/news/world-us-canada-64149300?page=all', 'https://www.bbc.com/news/world-europe-64146813?page=all', 'https://www.bbc.com/sport/live/football/64149966?page=all', 'https://www.bbc.comhttps://www.bbc.com/future/article/20221124-the-people-living-in-multiple-timelines?page=all']
['https://www.bbc.com/news/world-middle-east-64150409?page=all', 'https://www.bbc.com/news/world-us-canada-64149300?page=all', 'https://www.bbc.com/news/world-europe-64146813?page=all', 'https://www.bbc.comhttps://www.bbc.com/future/article/20221124-the-people-living-in-multiple-timelines?page=all']
['Palestinians have condemned a visit to a contested holy site in Jerusalem by a far right Israeli minister as an  "unprecedented provocation". ']
['Palestinians have condemned a visit to a contested holy site in Jerusalem by a far right Israeli minister as an  "unprecedented provocation". ', 'American football star Damar Hamlin is in 

ConnectionError: HTTPSConnectionPool(host='www.bbc.comhttps', port=443): Max retries exceeded with url: //www.bbc.com/future/article/20221124-the-people-living-in-multiple-timelines?page=all (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff3e915bc40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [5]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

,0,1,2
after,0.000000,0.218597,0.000000
american,0.000000,0.218597,0.000000
an,0.236180,0.000000,0.000000
arrest,0.000000,0.218597,0.000000
as,0.236180,0.000000,0.000000
attacks,0.000000,0.000000,0.246141
bid,0.000000,0.000000,0.246141
by,0.236180,0.000000,0.000000
campaign,0.000000,0.000000,0.246141
cardiac,0.000000,0.218597,0.000000


In [7]:
def get_similar_articles(q, df):
  import numpy as np
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
    # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
    # Calculate the similarity
  for i in range(3):
     sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    
    # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similarity Value:", v)
      print(documents_clean[k])        
      print()
  # Add The Query


q1 = input(str)
  # Call the function
get_similar_articles(q1, df)

query: a
The following are articles with the highest cosine similarity values: 


In [8]:
import requests
from bs4 import BeautifulSoup
def web(page,WebUrl):
     if(page>0):
          url = WebUrl
          code = requests.get(url)
          plain = code.text
          s = BeautifulSoup(plain, "html.parser")
          for link in s.findAll("a", {"class":"s-access-detail-page"}):
               tet = link.get("title")
               print(tet)
               tet_2 = link.get("href")
               print(tet_2)
web(1,"https://www.amazon.in/mobile-phones/b?ie=UTF8&node=1389401031&ref_=nav_shopall_sbc_mobcomp_all_mobiles")

In [9]:
import scrapy 
class spider1(scrapy.Spider):
        name = "Wikipedia"
        start_urls = ["https://en.wikipedia.org/wiki/Battery_(electricity)"]       
        def parse(self, response):
           pass

ImportError: cannot import name 'ParamSpec' from 'typing_extensions' (/Users/narin/opt/anaconda3/lib/python3.8/site-packages/typing_extensions.py)

In [10]:
import requests_html
from rich import console
import webbrowser

ses = requests_html.HTMLSession()
language = 'en'
console = console.Console()

search_term = console.input("Insert the search term: ")
url = f'https://{language}.wikipedia.org/w/index.php?search={search_term}&fulltext=1&ns0=1'
res = ses.get(url)
results = res.html.find('.mw-search-result')
urls = []
for num,result in enumerate(results):
    title = result.find('.mw-search-result-heading', first=True).text
    summary = result.find('.searchresult', first=True).text
    data = result.find('.mw-search-result-data', first=True).text
    urls.append(result.find('.mw-search-result-heading', first=True).absolute_links.pop())
    console.print(f'[blue]{num}\t{title}\n\t[white]{summary}\n\t[green]{data}\n')
selected_item = int(console.input("Select the number of the article you want to open: "))
webbrowser.open(urls[selected_item])

ModuleNotFoundError: No module named 'requests_html'